In [4]:
import os
import tiktoken

def count_tokens_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    encoding = tiktoken.encoding_for_model("text-embedding-ada-002")  # You can change the model name if needed
    tokens = encoding.encode(text)
    return len(tokens)

def main(folder_path):
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            num_tokens = count_tokens_in_file(file_path)
            results.append(num_tokens)
            # print(f"File: {filename}, Tokens: {num_tokens}")
    
    return results

# Replace 'path_to_your_folder' with the actual path to the folder containing your text files
folder_path = './actions'
token_counts = main(folder_path)

In [47]:
import numpy as np
mean_action = np.mean(token_counts)
print(f'On average a document contains {mean_action} tokens')

On average a document contains 568.2079207920792 tokens


In [48]:
cost_gpt3_input_per_token = 0.5 / 1_000_000
cost_gpt3_output_per_token = 1.5 / 1_000_000
cost_embedding_model = 0.02 / 1_000_000

print(f"Cost for Embedding SFI24 for Vector Database is ${np.sum(token_counts) * cost_embedding_model}")
print()
print(f"Cost to input 1 document to LLM is ${mean_action * cost_gpt3_input_per_token}")
print(f"Assuming top-3 docs are sent, cost per input is now ${(mean_action * cost_gpt3_input_per_token) * 3}")

Cost for Embedding SFI24 for Vector Database is $0.00114778

Cost to input 1 document to LLM is $0.0002841039603960396
Assuming top-3 docs are sent, cost per input is now $0.0008523118811881189


In [50]:
print('A Detailed breakdown of a query as a repsonse is approx 700 tokens')
cost_initial_response = cost_gpt3_output_per_token * 700

print(f'Cost for initial response is approximately ${cost_initial_response}')
print()

cost_to_serve_1query =((mean_action * cost_gpt3_input_per_token) * 3) + cost_initial_response

print(f'The Cost for to serve for a query answer pair is ${cost_to_serve_1query}')

A Detailed breakdown of a query as a repsonse is approx 700 tokens
Cost for initial response is approximately $0.00105

The Cost for to serve for a query answer pair is $0.0019023118811881188


If you require a follow-up question, You have two options:
- Stateless Query where each question is answered independly (cost is as above, but not recommended)
- Stateful Query where chat history is sent to the LLM (more useful)



In [39]:
print('If one additional followup query is needed, the previous answer gets sent as an input')

second_round_input_tokens = (mean_action * 3) + 700

print(f'Second round input now costs {second_round_input_tokens * cost_gpt3_input_per_token}')
print(f'Second round output still costs $ {cost_initial_response}')

cost_to_serve_2query = cost_to_serve_1query + (second_round_input_tokens * cost_gpt3_input_per_token) + cost_initial_response
print(f'Cost to serve for 2 queries is ${cost_to_serve_2query}')

If 1 followup query needed
Second round input now costs 0.0012023118811881187
Second round output still costs 0.00105
Cost to serve for 2 queries is $0.004154623762376237


There are about 1,500 users of find a day who spend about 6mins on the site. Let's assume they all ask 2 queries to the chatbot



In [51]:
daily_cost_to_serve_2queries = cost_to_serve_2query * 1500
print(f"In OpenAI API costs alone, running the service costs ${daily_cost_to_serve_2queries} per day")
print(f'This equates to ${daily_cost_to_serve_2queries * 365} a year (assuming no new users)')

In OpenAI API costs alone, running the service costs $6.231935643564356 per day
This equates to $2274.65650990099 a year (assuming no new users)


In [46]:
print(f'Doing Semantic search on an embedded database will cost ${np.sum(token_counts) * cost_embedding_model} if you need to re-embed the entire document set (unlikely)')

print(f'More likely, it will cost approx ${mean_action * cost_embedding_model:.8f} to add or change a document')

Doing Semantic search on an embedded database will cost $0.00114778 if you need to re-embed the entire document set (unlikely)
More likely, it will cost approx $0.00001136 to add or change a document


In [63]:
llm_energy_per_token = 0.00002
electricity_carbon_intensity = 0.2 #(https://www.itpenergised.com/new-uk-grid-emissions-factors-2023/#:~:text=The%20generation%20emissions%20factor%20for,year%20increase%20of%207%25).)

print('Carbon Footprint')
print(f'Lets assume ChatGPT costs {llm_energy_per_token:.5f} kWh per token (got from chatGPT- need to fact check)')
print(f'Average US carbon intensity for electricty is {electricity_carbon_intensity} kg CO2e per kWh.')

tokens_per_user = (mean_action * 3) + 700 + (mean_action * 3) + 700 + 700
print(f'Assuming our 2-response query which requires {tokens_per_user} tokens per user')


users_per_year = 1500 * 365
total_tokens_per_year = users_per_year * tokens_per_user
print(f'Expand that to our {users_per_year} yearly users, equals {total_tokens_per_year} tokens per year')
print()


total_kwh = total_tokens_per_year * llm_energy_per_token
print(f'Giving an energy usage of {total_kwh} kWh')
print(f'Giving Carbon Emmisions of {total_kwh * electricity_carbon_intensity}kg of Carbon a Year')

Carbon Footprint
Lets assume ChatGPT costs 0.00002 kWh per token (got from chatGPT- need to fact check)
Average US carbon intensity for electricty is 0.2 kg CO2e per kWh.
Assuming our 2-response query which requires 5509.247524752475 tokens per user
Expand that to our 547500 yearly users, equals 3016313019.80198 tokens per year

Giving an energy usage of 60326.26039603961 kWh
Giving Carbon Emmisions of 12065.252079207923kg of Carbon a Year


In [70]:
print('Alternative Carbon Footprint Calculation')
kg_carbon_per_query = 0.005 #this is the high end, but is probably conservative given we are doing RAG
print(f'Carbon emission per query is {kg_carbon_per_query}kg')
print(f'Across the {users_per_year} users (using 2 querues) that equates to {(users_per_year * 2) * kg_carbon_per_query}kg of Carbon per Year')


Alternative Carbon Footprint Calculation
Carbon emission per query is 0.005kg
Across the 547500 users (using 2 querues) that equates to 5475.0kg of Carbon per Year
